In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing as mp
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.optim import Adam
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [2]:
trained_model_path = '/workdir/security/home/junjiehuang2468/paper/trained_models_weight/ember/'
best_trained_model = '2022-01-18 14:55/2w_epoch:13_test_acc:0.896058.pt'
data_path = "/workdir/security/home/junjiehuang2468/paper/data/ember2018/"
train_data_path = data_path + "malwares/" 
train_label_path = data_path + "train_labels.csv" 

In [3]:
CUDA = True if torch.cuda.is_available() else False
NUM_WORKERS = 24
BATCH_SIZE = 80
LEAVE_BIT_NUMBER = 20000
KERNEL_SIZE = 500

In [4]:
LEAVE_BIT_NUMBER_DICT = {10000:'1w',20000:'2w'}

In [5]:
trainset = pd.read_csv(data_path + 'train_dataset.csv')
validset = pd.read_csv(data_path + 'valid_dataset.csv')
validset = validset.iloc[np.argwhere(validset['labels'].values == 1).squeeze(),:]

In [6]:
class ExeDataset(Dataset):
    def __init__(self, malware_names, data_path, labels, leave_bit_num):
        self.malware_names = malware_names
        self.data_path = data_path
        self.labels = labels
        self.leave_bit_num = leave_bit_num

    def __len__(self):
        return len(self.malware_names)

    def __getitem__(self, idx):
        with open(self.data_path + self.malware_names[idx] + '.txt','rb') as fp:
            data = [bit+1 for bit in fp.read()[:self.leave_bit_num]]
            padding = [0]*(self.leave_bit_num-len(data))
            data = data + padding

        return np.array(data), np.array([self.labels[idx]])

In [7]:
train_dataset = ExeDataset(
    trainset["id"].tolist(), 
    train_data_path, 
    trainset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)
valid_dataset = ExeDataset(
    validset["id"].tolist(), 
    train_data_path, 
    validset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)

In [8]:
trainloader = DataLoader(
    dataset = train_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)
validloader = DataLoader(
    dataset = valid_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)

In [9]:
def mp_func(i,inpu,te,gr):
    cum_grad = 0
    for j,(inp,g,t) in enumerate(zip(inpu,gr,te)):
        if inp != 0: continue
        max_idx = np.argmin(g).tolist()
        org_max_idx = np.argmax(t).tolist()
        cum_grad += np.mean(g)
        if g[max_idx] > 0: continue
        te[j][org_max_idx] = 0
        te[j][max_idx] = 1
    return [i,te,cum_grad]

In [10]:
class Model(nn.Module):
    def __init__(self, data_length = 2e6, kernel_size = 500):
        super().__init__()
        self.embedding = nn.Embedding(257, 8, padding_idx=0)
        self.conv_layer_1 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
        # self.bn_1 = nn.BatchNorm1d(128)
        self.conv_layer_2 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
        self.pool_layer_2 = nn.MaxPool1d(data_length//kernel_size)
        self.fc_layer_3 = nn.Linear(128, 128)
        self.fc_layer_4 = nn.Linear(128, 2)

    def forward(self, input_, loss_fn):
        one_hot_x = F.one_hot(input_,num_classes=257).float()
        one_hot_x.requires_grad = True
        one_hot_x.retain_grad()
        batch_acc = []
        batch_grad = []
        for _ in range(9):
            x = one_hot_x @ self.embedding.weight
            x = torch.transpose(x, -1, -2)

            x_conv_1 = self.conv_layer_1(x[:,:4,:])
            x_conv_2 = torch.sigmoid(self.conv_layer_2(x[:,4:,:]))

            x = x_conv_1*x_conv_2
            del x_conv_1,x_conv_2
            x = self.pool_layer_2(x).squeeze()
            
            x = self.fc_layer_3(x)
            x = self.fc_layer_4(x)
            
            acc = torch.argmax(torch.softmax(x,dim=-1),dim=-1).float().mean()
            batch_acc.append(acc.tolist())
            
            mislead_labels = torch.cat((torch.ones(len(x)).reshape(-1,1),torch.zeros(len(x)).reshape(-1,1)),dim=1).cuda()
            loss = loss_fn(x,mislead_labels)
            loss.backward()
            
            padding_place_mask = (input_ == 0)
            shape = padding_place_mask.shape
            expand_padding_place_mask = padding_place_mask.unsqueeze(-1).expand(shape[0],shape[1],257)
            grad_min_idx = torch.argmin(one_hot_x.grad,dim=-1)
            grad_sez_mask = (one_hot_x.grad <= 0)
            grad_mask = (expand_padding_place_mask & grad_sez_mask)
            batch_grad.append(torch.div(
                input=(grad_mask.float()*one_hot_x.grad).sum(),
                other=grad_mask.float().sum()
            ).detach().cpu().numpy())
            new_x = torch.argmin(grad_mask.float()*one_hot_x.grad,dim=-1)
            grad_all_gz_mask = grad_sez_mask.sum(dim=-1) == 257
            mask = ((~grad_all_gz_mask) & padding_place_mask)
            temp = (1 - mask.float())*input_ + mask.float()*new_x
            one_hot_x.data = F.one_hot(temp.long(),num_classes=257).float()
            
            # one_hot_x.grad.zero
            
        x = one_hot_x @ self.embedding.weight
        x = torch.transpose(x, -1, -2)

        x_conv_1 = self.conv_layer_1(x[:,:4,:])
        x_conv_2 = torch.sigmoid(self.conv_layer_2(x[:,4:,:]))

        x = x_conv_1*x_conv_2
        del x_conv_1,x_conv_2
        x = self.pool_layer_2(x).squeeze()

        x = self.fc_layer_3(x)
        x = self.fc_layer_4(x)
        return x.detach().cpu().numpy(),batch_acc,batch_grad

In [11]:
model = Model(data_length=LEAVE_BIT_NUMBER,kernel_size=KERNEL_SIZE)

ce_loss = nn.CrossEntropyLoss()
optim = Adam(model.parameters())

model = model.cuda() if CUDA else model
ce_loss = ce_loss.cuda() if CUDA else ce_less

In [12]:
model.load_state_dict(torch.load(trained_model_path + best_trained_model))

<All keys matched successfully>

In [ ]:
model.eval()
acc = []
preds = []
labels = []
total_batch_acc = []
total_batch_grad = []
bar = tqdm(validloader)
for step, (batch_data,batch_label) in enumerate(bar):
    batch_data = batch_data.cuda() if CUDA else batch_data
    batch_label = batch_label.cuda() if CUDA else batch_label
    batch_label = batch_label.squeeze()
    temp_label = torch.zeros((len(batch_label),2))
    for idx,target in enumerate(batch_label.squeeze()): temp_label[idx,target] = 1
    temp_label = temp_label.cuda() if CUDA else temp_label

#     pred,input_,one_hot_x = model(batch_data,ce_loss)
#     break
    pred,batch_acc,batch_grad = model(batch_data,ce_loss)
    pred = np.argmax(pred,1)
    batch_label = batch_label.cpu().data.numpy()
    
    preds.extend(pred.tolist())
    labels.extend(batch_label.tolist())
    
    temp_acc = (batch_label == pred).mean()
    acc.append(temp_acc)
    total_batch_acc.append(batch_acc + [temp_acc])
    total_batch_grad.append([0] + batch_grad)
    total_batch_acc_str = '[' + ' '.join(map(lambda x: '%.7f'%x,np.mean(total_batch_acc,axis=0))) + ']'
    total_batch_grad_str = '[' + ' '.join(map(lambda x: '%.7f'%x,np.mean(total_batch_grad,axis=0))) + ']'
    print(total_batch_acc_str)
    print(total_batch_grad_str)
    print(f"test：{temp_acc:4f}, test mean: {np.mean(acc):4f}")

  0%|          | 1/750 [00:03<46:10,  3.70s/it]

[0.8750000 0.0875000 0.0625000 0.0375000 0.0375000 0.0375000 0.0375000 0.0375000 0.0375000 0.0375000]
[0.0000000 -0.0000153 -0.0000193 -0.0000208 -0.0000209 -0.0000210 -0.0000210 -0.0000211 -0.0000212 -0.0000213]
test：0.037500, test mean: 0.037500


  0%|          | 2/750 [00:04<27:22,  2.20s/it]

[0.8687500 0.1187500 0.0625000 0.0500000 0.0437500 0.0437500 0.0375000 0.0375000 0.0375000 0.0375000]
[0.0000000 -0.0000145 -0.0000184 -0.0000194 -0.0000195 -0.0000196 -0.0000197 -0.0000198 -0.0000199 -0.0000200]
test：0.037500, test mean: 0.037500


  0%|          | 3/750 [00:05<21:13,  1.70s/it]

...

[0.8895428 0.1634241 0.0968872 0.0797665 0.0762646 0.0741245 0.0727626 0.0719844 0.0716926 0.0712062]
[0.0000000 -0.0000155 -0.0000184 -0.0000192 -0.0000193 -0.0000194 -0.0000194 -0.0000195 -0.0000196 -0.0000197]
test：0.075000, test mean: 0.071206


 34%|███▍      | 258/750 [04:24<08:20,  1.02s/it]

[0.8895349 0.1635659 0.0969961 0.0798450 0.0762597 0.0741279 0.0727713 0.0719961 0.0717054 0.0712209]
[0.0000000 -0.0000155 -0.0000184 -0.0000191 -0.0000193 -0.0000194 -0.0000194 -0.0000195 -0.0000196 -0.0000197]
test：0.075000, test mean: 0.071221


 35%|███▍      | 259/750 [04:25<08:18,  1.02s/it]

[0.8896236 0.1635618 0.0971525 0.0799710 0.0763996 0.0741795 0.0728282 0.0720077 0.0717181 0.0712355]
[0.0000000 -0.0000155 -0.0000184 -0.0000191 -0.0000193 -0.0000193 -0.0000194 -0.0000195 -0.0000196 -0.0000197]
test：0.075000, test mean: 0.071236


In [15]:
# padding_place_mask = (input_ == 0)
# shape = padding_place_mask.shape
# expand_padding_place_mask = padding_place_mask.unsqueeze(-1).expand(shape[0],shape[1],257)
# grad_min_idx = torch.argmin(one_hot_x.grad,dim=-1)
# grad_sez_mask = (one_hot_x.grad <= 0)
# grad_mask = (expand_padding_place_mask & grad_sez_mask)
# new_x = torch.argmin(grad_mask.float()*one_hot_x.grad,dim=-1)
# grad_all_gz_mask = grad_sez_mask.sum(dim=-1) == 257
# mask = ((~grad_all_gz_mask) & padding_place_mask)
# batch_data.data = (1 - mask.float())*input_ + mask.float()*new_x

# preds,input_,one_hot_x = model(batch_data.long(),ce_loss)

In [37]:
padding_place_mask = (input_ == 0)
shape = padding_place_mask.shape
expand_padding_place_mask = padding_place_mask.unsqueeze(-1).expand(shape[0],shape[1],257)

In [38]:
grad_min_idx = torch.argmin(one_hot_x.grad,dim=-1)

In [39]:
grad_sez_mask = (one_hot_x.grad <= 0)

In [40]:
grad_mask = (expand_padding_place_mask & grad_sez_mask)

In [41]:
new_x = torch.argmin(grad_mask.float()*one_hot_x.grad,dim=-1)

In [42]:
grad_all_gz_mask = grad_sez_mask.sum(dim=-1) == 257

In [43]:
mask = ((~grad_all_gz_mask) & padding_place_mask)

In [45]:
temp = (1 - mask.float())*input_ + mask.float()*new_x


In [48]:
one_hot_x.data = F.one_hot(temp.long(),num_classes=257).float()

In [24]:
# one_hot_x_max_idx = torch.argmax(one_hot_x,dim=-1)

In [28]:
# preds,input_,one_hot_x = model(batch_data.long(),ce_loss)

In [29]:
# _, predicted = torch.max(preds, 1)
# acc = (batch_label.cpu().data.numpy() == predicted.cpu().data.numpy()).mean()
# acc

0.08333333333333333